In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train_df = pd.read_csv("/content/data_description.txt")
test_df = pd.read_csv("/content/data_description.txt")
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

##  Exploratory Data Analysis (EDA)
We'll begin by checking the distribution of the target variable and identifying missing values.


In [ ]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")


In [ ]:
# Distribution of the target variable
sns.histplot(train_df['SalePrice'], kde=True)
plt.title("SalePrice Distribution")
plt.show()

# Log-transform if needed
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

# Check missing values
missing = train_df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing.head(20)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Log transform the target variable
log_saleprice = np.log1p(train_df['SalePrice'])

# Plot
sns.displot(log_saleprice, kde=True)
plt.title("Log-Transformed SalePrice Distribution")
plt.xlabel("Log(SalePrice)")
plt.show()

In [ ]:
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])
sns.histplot(train_df['SalePrice'],kde=True)
plt.show()

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import probplot

# Assuming train_df is already loaded and contains 'SalePrice'
# QQ Plot
probplot(train_df['SalePrice'], plot=plt)
plt.show()


In [ ]:
# Select only numerical columns from the training data
df_num = train_df.select_dtypes(include=[np.number])

# Compute correlation matrix
corr = df_num.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, vmax=1.0, square=True, cmap='coolwarm', annot=True)
plt.title("Correlation Matrix of Numerical Features")
plt.show()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);
plt.show()

### Handling Missing values

In [ ]:
# Combine train and test data for preprocessing
train_labels = train_df['SalePrice']
train_df = train_df.drop(['SalePrice'], axis=1)

# Concatenate train and test
all_data = pd.concat([train_df, test_df], axis=0)


In [ ]:
null_columns = all_data.columns[all_data.isna().any()]
columns_with_null =all_data[null_columns]
nullcounts = columns_with_null.isna().sum().sort_values(ascending=False)

# usually we ignore the columns when percentage of null values exceeds than 25%
null_percent = (nullcounts/len(all_data)) * 100
null_percent_df = pd.DataFrame({"Total":nullcounts,"Missing Ratio":null_percent})
null_percent_df

In [ ]:
#visualizing the missing value by percentage
plt.figure(figsize=(12,10))
plt.xticks(rotation=90)
sns.barplot(y=null_percent,x =null_percent.index )
plt.xlabel('Missing values ',fontsize=15)
plt.ylabel('Percent of Missing value',fontsize=15)
plt.title('Percentage of Missing value for each Features',fontsize = 30)
plt.show()

In [ ]:
# Handle categorical columns
categorical_cols = all_data.select_dtypes(include='object').columns
all_data[categorical_cols] = all_data[categorical_cols].fillna("NA")

# Handle numerical columns
numerical_cols = all_data.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    if all_data[col].isna().sum() > 0:
        all_data[col] = all_data[col].fillna(all_data[col].median())

### Model Training and Visualization

In [ ]:
import pandas as pd
import numpy as np

# Load the datasets
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")


In [ ]:
print(train_df.columns)  # See if 'SalePrice' is there


In [ ]:
# Store target and IDs
y = np.log1p(train_df['SalePrice'])  # log-transform SalePrice
test_ID = test_df['Id']

# Drop Id and SalePrice
X = train_df.drop(['Id', 'SalePrice'], axis=1)
X_test = test_df.drop(['Id'], axis=1)

# Combine for preprocessing
all_data = pd.concat([X, X_test], axis=0)

# Fill missing values
categorical_cols = all_data.select_dtypes(include='object').columns
numerical_cols = all_data.select_dtypes(include=['int64', 'float64']).columns

all_data[categorical_cols] = all_data[categorical_cols].fillna("NA")
all_data[numerical_cols] = all_data[numerical_cols].apply(lambda col: col.fillna(col.median()))

# One-hot encode
all_data = pd.get_dummies(all_data)

# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
all_data_scaled = pd.DataFrame(scaler.fit_transform(all_data), columns=all_data.columns)

# Split back to train and test
X_all = all_data_scaled.iloc[:len(y), :]
X_final_test = all_data_scaled.iloc[len(y):, :]


In [ ]:
# ==== STEP 4: One-Hot Encoding ====
all_data = pd.get_dummies(all_data)

# ==== STEP 5: Feature Scaling ====
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
all_data_scaled = pd.DataFrame(scaler.fit_transform(all_data), columns=all_data.columns)

# ==== STEP 6: Re-split into train and test ====
X_all = all_data_scaled.iloc[:len(y), :]
X_final_test = all_data_scaled.iloc[len(y):, :]

# ==== STEP 7: Train/Validation Split ====
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y, test_size=0.2, random_state=42)

# ==== STEP 8: Model Evaluation Function ====
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

def model_evaluation(y_test, y_pred, tolerance=0.1):
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    within_tolerance = np.abs(y_test - y_pred) <= (tolerance * y_test)
    accuracy = np.mean(within_tolerance) * 100

    print("\n📊 Model Evaluation Summary")
    print("--------------------------------------")
    print(f" R² Score       : {r2:.4f}")
    print(f" MSE            : {mse:.4f}")
    print(f" MAE            : {mae:.4f}")
    print(f" RMSE           : {rmse:.4f}")
    print(f" Accuracy (±{int(tolerance * 100)}%) : {accuracy:.2f}%")
    print("--------------------------------------")


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# No tuning, just 1st-degree poly features
best_linear_model = Pipeline([
    ('poly', PolynomialFeatures(degree=1, include_bias=False)),
    ('linreg', LinearRegression(fit_intercept=True))
])

best_linear_model.fit(X_train, y_train)
y_pred_valid = best_linear_model.predict(X_valid)

# Evaluation
model_evaluation(y_valid, y_pred_valid)


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge, Lasso

# Lighter set of base learners
estimators = [
    ('ridge', Ridge(alpha=10)),
    ('lasso', Lasso(alpha=0.001, max_iter=10000)),
    ('linear', best_linear_model)
]

stack = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=5),
    n_jobs=-1
)

stack.fit(X_train, y_train)
y_pred_valid_stack = stack.predict(X_valid)

# Evaluation
print("\n Stacking Regressor Performance:")
model_evaluation(y_valid, y_pred_valid_stack)


### Submission

In [ ]:
# Predict on the final test set
y_pred_test = stack.predict(X_final_test)

# Convert from log scale back to actual prices
y_pred_test = np.expm1(y_pred_test)

# Prepare submission
submission = pd.DataFrame({'Id': test_ID, 'SalePrice': y_pred_test})
submission.to_csv('submission.csv', index=False)
print(" Submission file saved as 'submission.csv'")
